# Homework: Financial Ratio Quantile Strategy
Robert Hatem

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy as sp
import quandl
import functools
import seaborn as sns
import time
%matplotlib inline

In [2]:
# API key
quandl.ApiConfig.api_key = "wFcUAbwJv3FbxzDwExsY"

In [3]:
# for quandl data
@functools.lru_cache(maxsize=16)
def fetch_quandl(my_securities, start_date="2011-01-01", end_date="2018-01-01"):
    #print("Fetching from Quandl")
    if len(my_securities) != 1:
        qdata = quandl.get(list(my_securities), start_date="2010-12-01", end_date="2018-01-01", returns="pandas")
    elif len(my_securities) == 1:
        my_securities = my_securities[0]
        qdata = quandl.get(list(my_securities), start_date="2010-12-01", end_date="2018-01-01", returns="pandas")
    return qdata

## Read in master files
I used the "bulk download" from Quandl.

In [4]:
data1 = pd.read_csv('ZFB-complete/ZFB-1.csv', delimiter='|', low_memory=False)
data2 = pd.read_csv('ZFB-complete/ZFB-2.csv', delimiter='|', low_memory=False)
data3 = pd.read_csv('ZFB-complete/ZFB-3.csv', delimiter='|', low_memory=False)
data4 = pd.read_csv('ZFB-complete/ZFB-4.csv', delimiter='|', low_memory=False)
data5 = pd.read_csv('ZFB-complete/ZFB-5.csv', delimiter='|', low_memory=False)
data6 = pd.read_csv('ZFB-complete/ZFB-6.csv', delimiter='|', low_memory=False)

In [5]:
# number of tickers in data3
np.unique(data3['TICKER'].values).shape

(2372,)

ETF price and returns from Quandl

In [6]:
eod_etf = quandl.get(("EOD/SPY"), start_date="2010-12-01", end_date="2018-01-01", returns="pandas")
eod_etf1 = eod_etf.loc[:,['Adj_Close']]
eod_etf1['etf_returns'] = eod_etf1['Adj_Close'].pct_change()
eod_etf1.drop(columns=['Adj_Close'], inplace=True)
eod_etf1 = eod_etf1.reset_index()

Collect the universe of stocks by looking through all tickers in the 6 files and filtering out the bad ones.

In [10]:
cols = ['TICKER',
        'PER_END_DATE',
        'PER_TYPE',
        'ZACKS_SECTOR_CODE',
        'ZACKS_X_IND_CODE',
        'FILING_DATE',
        'NBR_SHARES_OUT',
        'TOT_DEBT_TOT_EQUITY', # debt to market cap
        'BOOK_VAL_PER_SHARE', # price to book
        'DILUTED_NET_EPS']    # price to earnings

def clean_data():
    dic_output = {}
    for data in [data1, data2, data3, data4, data5, data6]:
        df = data.loc[(data.ZACKS_SECTOR_CODE!=5) & (data.ZACKS_SECTOR_CODE!=13)].copy()  # filter out wrong sectors
        df1 = df.loc[df.PER_TYPE == 'Q'].copy()  # filter out annual data
        tickers = np.unique(df1['TICKER'].values)
        for i, ticker in enumerate(tickers):
            if i%(data.shape[0]/100)==0:
                print(data.shape[0])
                print('{} of {}'.format(i,data.shape[0]))
            ticker1 = ticker.replace('.','')
            df2 = df1.loc[df1.TICKER==ticker1, cols].copy()
            #if df2.loc[:,'ZACKS_SECTOR_CODE'].iloc[-1] in [5,13]:  # if wrong sector
            #    continue
            #elif df2.loc['ZACKS_X_IND_CODE'].iloc[-1] in [5,13]:
            #    continue
            num_above_1 = sum(df2.TOT_DEBT_TOT_EQUITY > 0.1)==0
            #if i==100:
            #    return dic_output
        
            if sum(df2.TOT_DEBT_TOT_EQUITY > 0.1) == 0:
                #print(i,'| Not enough debt | {}'.format(ticker))
                continue
            df2.drop(columns=['ZACKS_SECTOR_CODE', 'ZACKS_X_IND_CODE'], inplace=True)
            df2 = df2[df2.PER_TYPE=='Q']
            df2.PER_END_DATE = pd.to_datetime(df2.PER_END_DATE)
            df3 = df2.loc[(df2.PER_END_DATE >= '2010-10-30') & (df2.PER_END_DATE < '2018-04-01')].copy()
            df3['TOT_DEBT_TOT_EQUITY'].fillna(0, inplace=True)
            quandl_input = 'EOD/'+ticker+'.11'
            #print(quandl_input)
            try:
                df4 = quandl.get(quandl_input, start_date="2011-01-01", end_date="2018-01-01", returns="pandas")  # adjusted close price
            except:
                #print(i,'| No price data | {}'.format(ticker))
                continue
            df4 = df4.reset_index()
            df4.Date = pd.to_datetime(df4.Date)
            df3.PER_END_DATE = pd.to_datetime(df3.PER_END_DATE)
            try:
                df5 = df4.merge(df3, left_on='Date', right_on='PER_END_DATE', how='left').merge(eod_etf1, left_on='Date', right_on='Date', how='left')
            except:
                continue
            df5.fillna(method='ffill', inplace=True)
            df5['stock_returns'] = df5['Adj_Close'].pct_change()
            df5['debt_to_mktcap'] = df5['TOT_DEBT_TOT_EQUITY'].values.copy()
            df5['price_to_book'] = df5['Adj_Close']/df5['BOOK_VAL_PER_SHARE']   # compute new columns
            df5['price_to_earnings'] = df5['Adj_Close']/df5['DILUTED_NET_EPS']
            df5.drop(columns=['Adj_Close','PER_TYPE','PER_END_DATE', 'FILING_DATE',	'NBR_SHARES_OUT', 'TOT_DEBT_TOT_EQUITY', 'BOOK_VAL_PER_SHARE', 'DILUTED_NET_EPS'], inplace=True)
            df5['TICKER'].fillna(method='bfill', inplace=True)
            if df5.shape[0] < 1400:  # 
                #print(i,'| Not enough data | {}'.format(ticker))
                continue
            if df5.isna().sum(axis=0)[5] > 500:
                #print(i,'Too many NaNs | {}'.format(ticker))
                continue
            #print(i,'| adding | {}'.format(ticker), time.strftime('%l:%M%p %z on %b %d, %Y'))
            dic_output[ticker]=df5
    return dic_output

df5 = clean_data()

85459
0 of 85459
84386
0 of 84386
85623
0 of 85623
86366
0 of 86366
87129
0 of 87129
44647
0 of 44647


Remove tickers that contain columns with all NaNs (ratios not available)

In [213]:
# remove tickers that columns with all NaNs
dict1 = {}
for key, value in df5.items():
    if sum(value.isna().sum() / value.shape[0] > 0.9) > 0:
        continue
    else:
        dict1[key]=value

In [214]:
# number of tickers
len(dict1)

1839

Concatenate all tickers into one dataframe.

In [225]:
df_master = dict1['A'].copy()
keys = list(dict1.keys())

for i, key in enumerate(keys):
    if key=='A':
        continue
    else:
        df_master = pd.concat([df_master, dict1[key]])
    if i%100==0: print(i, 'of', len(keys))
df_master.shape

100 of 1839
200 of 1839
300 of 1839
400 of 1839
500 of 1839
600 of 1839
700 of 1839
800 of 1839


/Users/roberthatem/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


900 of 1839
1000 of 1839
1100 of 1839
1200 of 1839
1300 of 1839
1400 of 1839
1500 of 1839
1600 of 1839
1700 of 1839
1800 of 1839


(3218010, 9)

In [230]:
df_master.head()

,Date,TICKER,alpha_0,alpha_1,debt_to_mktcap,etf_returns,price_to_book,price_to_earnings,stock_returns
0,2011-01-03,A,NaN,NaN,NaN,0.010338,NaN,NaN,NaN
1,2011-01-04,A,NaN,NaN,NaN,-0.000551,NaN,NaN,-0.009312
2,2011-01-05,A,NaN,NaN,NaN,0.005198,NaN,NaN,-0.002169
3,2011-01-06,A,NaN,NaN,NaN,-0.001959,NaN,NaN,0.001932
4,2011-01-07,A,NaN,NaN,NaN,-0.001962,NaN,NaN,0.003375


Drop the erroneous columns for the regression alphas (with ETF returns)

In [258]:
df_master.drop(columns=['alpha_0', 'alpha_1'], inplace=True)

In [264]:
# sort by date then ticker
# drop rows with NaN
d1 = df_master.sort_values(by=['Date', 'TICKER']).dropna()

In [268]:
d1['Date_new'] = pd.DatetimeIndex(d1.Date)

In [300]:
d2 = d1.set_index(pd.DatetimeIndex(d1['Date'])).drop(columns=['Date', 'Date_new'])
d2['year'] = d2.index.year
d2['month'] = d2.index.month
d2['day'] = d2.index.day

In [309]:
# first day of the month
d2[(d2.year==2011) & (d2.month==2)].day.min()

1

Go long the top 10 tickers at the start of each month. See their performance over that month.

In [338]:
# find first date of each month
months = [i for i in range(1,13)]
years = [2012,2013,2014,2015,2016,2017]
first_day_of_month = {}
for year in years:
    for month in months:
        first_day_of_month[(year, month)] = d2[(d2.year==year) & (d2.month==month)].day.min()
len(first_day_of_month)

72

In [341]:


for key,value in first_day_of_month.items():
    first_day = value
    d3 = d2[(d2.year==key[0]) & (d2.month==key[1]) & (d2.day==3)]
    
    break
    

In [ ]:
month_start=(EOD_Data[[]].assign(date=EOD_Data[[]].index).resample("BMS").first().values.flatten())[1:]
print(month_start.shape)
month_end=EOD_Data[[]].assign(date=EOD_Data[[]].index).resample("M").last().values.flatten()[1:]
print(month_end.shape)

In [346]:
d2.resample('M').first()

,TICKER,debt_to_mktcap,etf_returns,price_to_book,price_to_earnings,stock_returns,year,month,day
Date,,,,,,,,,
2011-01-31,A,0.6406,0.007516,2.909033,52.137947,0.020742,2011,1,31
2011-02-28,A,0.6406,0.016009,2.924332,52.412160,0.005259,2011,2,1
2011-03-31,A,0.6406,-0.016673,2.829057,50.704558,-0.033270,2011,3,1
2011-04-30,A,0.6406,0.004224,3.139224,56.263612,0.008039,2011,4,1
2011-05-31,A,0.6406,-0.001539,3.507808,62.869664,0.010619,2011,5,2
2011-06-30,A,0.6406,-0.022461,3.338120,59.828387,-0.037497,2011,6,1
2011-07-31,A,0.6406,0.014776,3.616297,64.814086,0.017413,2011,7,1
2011-08-31,A,0.6406,-0.004253,2.836707,50.841665,-0.032495,2011,8,1
2011-09-30,A,0.6406,-0.010473,2.492463,44.671862,-0.027936,2011,9,1


In [105]:
from sklearn.linear_model import LinearRegression

In [194]:
dict2 = dict1.copy()
keys = list(dict2.keys())
counter = 0

# compute betas
for key, d in dict2.items():
    print(key, counter)
    d1 = d.loc[:,['Date','stock_returns','etf_returns']].copy()
    d1['alpha_0'] = np.nan
    d1['alpha_1'] = np.nan
    
    for index, row in d.iterrows():
        # find dates corresponding to the previous month
        month_previous = row['Date'] - pd.DateOffset(months=1)
        
        d2 = d1.loc[(d1.Date >= month_previous) & (d1.Date < row['Date'])].copy() # filter to past month
        if d2.shape[0]<2: continue
        if sum(d2.loc[:,['stock_returns','etf_returns']].isna().sum() ==1): continue
        X = d2['etf_returns'].values.reshape(-1,1)
        y = d2['stock_returns'].values.reshape(-1,1)
        reg = LinearRegression().fit(X, y)
        alpha_0, alpha_1 = reg.intercept_[0], reg.coef_[0][0]
        d1.at[index,'alpha_0'] = alpha_0
        d1.at[index,'alpha_1'] = alpha_1
        #if index==30: break
    d['alpha_0'] = d1['alpha_0'].values
    d['alpha_1'] = d1['alpha_1'].values
    counter +=1
    

A 0
AAL 1
AAON 2
AAP 3
AAPL 4
AAWW 5
ABB 6
ABC 7
ABG 8
ABM 9
ABT 10
ACAD 11
ACCO 12
ACHC 13
ACHN 14
ACLS 15
ACM 16
ACOR 17
ACRX 18
ACU 19
ADBE 20
ADES 21
ADI 22
ADM 23
ADP 24
ADS 25
ADSK 26
ADTN 27
ADUS 28
ADXS 29
ADYX 30
AE 31
AEHR 32
AEIS 33
AEM 34
AEP 35
AES 36
AETI 37
AEY 38
AGCO 39
AGEN 40
AHC 41
AHPI 42
AIMC 43
AIN 44
AIR 45
AIRT 46
AIT 47
AKAM 48
AKRX 49
AKS 50
AL 51
ALB 52
ALCO 53
ALG 54
ALGN 55
ALGT 56
ALIM 57
ALK 58
ALKS 59
ALNY 60
ALOT 61
ALSK 62
ALXN 63
AMAG 64
AMAT 65
AMCX 66
AMD 67
AME 68
AMED 69
AMGN 70
AMID 71
AMKR 72
AMN 73
AMPE 74
AMRC 75
AMRS 76
AMS 77
AMSC 78
AMWD 79
AMZN 80
ANDE 81
ANF 82
ANGI 83
ANGO 84
ANIK 85
ANIP 86
ANSS 87
ANTH 88
ANTM 89
AOSL 90
AP 91
APA 92
APD 93
APDN 94
APH 95
APOG 96
APU 97
APYX 98
ARAY 99
ARC 100
ARCB 101
ARCI 102
ARCW 103
ARDMQ 104
AREX 105
ARLP 106
ARNA 107
ARNC 108
ARQL 109
ARRY 110
ARTNA 111
ARTW 112
ARW 113
ASCMA 114
ASGN 115
ASH 116
ASTE 117
ASYS 118
AT 119
ATEC 120
ATNI 121
ATR 122
ATRC 123
ATRI 124
ATRO 125
ATSG 126
ATU 127
ATVI 

KeyboardInterrupt: 